In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
feature_data_dicts = SETTING["feature_data"]

if feature_data_dicts is None:

    feature_data_dicts = {}

for data_name, data_dict in feature_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
sample_data_dicts = SETTING["sample_data"]

if sample_data_dicts is None:

    sample_data_dicts = {}

for data_name, data_dict in sample_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, data_dicts, factor_x_element in (
    ("w", feature_data_dicts, w.T),
    ("h", sample_data_dicts, h),
):

    if SETTING["{}_summary".format(w_or_h)] is None:

        continue

    for factor_name, data_name_indices in SETTING["{}_summary".format(w_or_h)].items():

        data_dicts_ = {}

        score_moe_p_value_fdr_dicts = {}

        for data_name, indices in data_name_indices.items():

            data_dicts_[data_name] = {
                "dataframe": data_dicts[data_name]["dataframe"].loc[indices]
            }

            score_moe_p_value_fdr_dicts[data_name] = pd.read_csv(
                os.path.join(
                    PATH["h|match/"],
                    factor_name,
                    "{}.tsv".format(ccal.normalize_file_name(data_name)),
                ),
                sep="\t",
                index_col=0,
            )

        if SETTING["factor_alias"] and factor_name in SETTING["factor_alias"]:

            title = SETTING["factor_alias"][factor_name]

        else:

            title = factor_name

        ccal.make_summary_match_panel(
            factor_x_element.loc[factor_name],
            data_dicts_,
            score_moe_p_value_fdr_dicts,
            plot_only_shared_by_target_and_all_data=SETTING[
                "plot_only_shared_by_target_and_all_data"
            ],
            plot_std=SETTING["plot_std"],
            title=factor_name,
            html_file_path=os.path.join(
                PATH["h|summary/"], "{}.html".format(factor_name)
            ),
        )